In [1]:
import numpy as np
import matplotlib.pyplot as plt

from rsbp import RSBP, EntropicRSBP
from rsbp import exact_rsbp, exact_entreg_rsbp
from rsbp import calc_B, calc_f_rsbp, calc_g_rsbp
from sinkhorn_rsbp import calc_U
from utils import norm_inf

import time

In [2]:
# Dimension
n = 10
m = 2

# Regularization
tau = np.float64(1.0)

In [3]:
np.random.seed(3698)

# Cost matrix
C = np.random.uniform(low=0.01, high=0.1, size=(m, n, n)).astype(np.float64)
# C = (C + C.T) / 2.0

# Marginal vectors
p = np.random.rand(m, n).astype(np.float64)
p = p / p.sum(1, keepdims=True)

# weight
w = np.random.rand(m).astype(np.float64)
w = w / w.sum()

# Theory checking

In [13]:
# Original UOT problem
rsbp = RSBP(C, p, w, tau)

# Optimal solution
_, X_hat = exact_rsbp(rsbp)

print('Optimal:', calc_f_rsbp(rsbp, X_hat))

Optimal: 0.019808113478932632


In [14]:
eps = 0.01

# Entropic regularization parameter
U = calc_U(rsbp, eps)
eta = eps / U

# Convert to Entropic Regularized UOT
ersbp = EntropicRSBP(C, p, w, tau, eta)

In [15]:
_, u_star, v_star = exact_entreg_rsbp(ersbp)
X_star = calc_B(ersbp, u_star, v_star)
print('Optimal:', calc_f_rsbp(rsbp, X_star), calc_g_rsbp(ersbp, X_star))

Optimal: 0.01975416166417018 0.014712280538882197


In [17]:
f_optimal = calc_f_rsbp(rsbp, X_hat)

In [ ]:
# print(f'Epsilon:', eps)
# start = time.time()

# # Sinkhorn
# _, log = robust_semisinkhorn_eps(ersot, f_optimal, eps, 
#                                     patience=1000, verbose=True)

# print('Time elapsed:', time.time() - start)

In [ ]:
# plt.plot(log['f'][1:])
# plt.axhline(f_optimal, color='red')
# plt.show()

In [ ]:
# y = []

# for uk, vk in zip(log['u'], log['v']):
#     Xk = calc_B(ersot, uk, vk)
#     # y.append(norm_inf(Xk - X_hat))
#     y.append(calc_g_rsot(ersot, Xk))

# plt.plot(y)